### Variables

In [21]:
SHOW_BOROUGHS_BOUNDARIES = False
SHOW_COMMUNITY_DISTRICT_BOUNDARIES = True
SHOW_FIRE_BATALLIONS = False
SHOW_FIREHOUSES = True

### Create Map

In [22]:
import folium

map = folium.Map(location=[40.71552394800378, -74.01878975440212], zoom_start=11, tiles='CartoDB positron')

### Read Incidents Json

In [23]:
import pandas as pd

incidents = pd.read_json('/Users/loic.doerr/dev/ny-emergency-ai/data/fire-incidents/interims.json')

### Read Boroughs Json

In [24]:
import json

with open('../data/geo-data/boroughs.geojson') as rawData:
  boroughs_boundaries_geo = json.load(rawData)

### Draw Borough Boundaries

In [25]:
if SHOW_BOROUGHS_BOUNDARIES:
  folium.GeoJson(
    boroughs_boundaries_geo,
    style_function=lambda feature: {
      'fillColor': 'green' if feature['properties']['boro_name'] == 'Manhattan' else 'blue' if feature['properties']['boro_name'] == 'Brooklyn' else 'red' if feature['properties']['boro_name'] == 'Bronx' else 'orange' if feature['properties']['boro_name'] == 'Queens' else 'purple',
      'color': 'black',
      'weight': 2,
      'fillOpacity': 0.2,
    }
  ).add_to(map)

### Read Community Districts Json

In [26]:
with open('/Users/loic.doerr/dev/ny-emergency-ai/data/geo-data/community-districts.geojson') as rawData:
  community_districts_geo = json.load(rawData)

### Calculate Incident Denisty

In [27]:
incident_density = incidents['communitydistrict'].value_counts(normalize=True)

max_density = incident_density.max()

### Draw Community Districts Boundaries

In [28]:
if SHOW_COMMUNITY_DISTRICT_BOUNDARIES:
  color_map = folium.LinearColormap(['green', 'red'], vmin=0, vmax=max_density)

  def calculate_opacity(feature):
    district_id = int(feature['properties']['boro_cd'])
    density = incident_density.get(district_id, 0)
    opacity = density / max_density
    return {'fillColor': color_map(opacity), 'color': 'black', 'weight': 2, 'fillOpacity': opacity}


  folium.GeoJson(
    community_districts_geo,
    style_function=calculate_opacity
  ).add_to(map)

### Read Fire Battalions Json

In [29]:
with open('../data/geo-data/fire-battalions.geojson') as rawData:
  fire_battalions = json.load(rawData)

### Draw Fire Battalions

In [30]:
if SHOW_FIRE_BATALLIONS:
  folium.GeoJson(fire_battalions, style_function=lambda function: {'fillColor': None, 'color': 'red', 'weight': 2, 'fillOpacity': 0}).add_to(map)

### Draw Firehouse Markers

In [31]:
if SHOW_FIREHOUSES:
  locations = pd.read_csv('/Users/loic.doerr/dev/ny-emergency-ai/data/locations/FDNY_Firehouse_Listing_20240603.csv')
  for index, location in locations.iterrows():
    lat = location['Latitude']
    lng = location['Longitude']
    folium.Marker([lat, lng], popup=location['FacilityAddress']).add_to(map)

### Save Map

In [32]:
map.save('../maps/districts.html')